<div style="display: inline-block;">
    <img src="images/nhsa_logo.png" alt="Image" style="text-align: left;">
</div>

# Parent Gauge Data Analysis Project
---
## Data Wrangling Script and Documentation

In this script, we will provide a step-by-step demonstration of how script is being cleaned. 

In [15]:
# Start with the necessary imports
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from tabulate import tabulate
from prettytable import PrettyTable
from rich.console import Console
from rich.table import Table
from IPython.display import display

# uses old version of google trans: pip3 install googletrans==3.1.0a0
from googletrans import Translator

# Utility function for gender imputation - DISCLAIMER: Sensitivity and Accuracy Considerations
# This function is intended for filling in missing gender values for statistical purposes only.
# Please note that gender imputation methods may not accurately reflect an individual's gender identity.
# Use caution and sensitivity when interpreting or applying these imputed values.
import gender_guesser.detector as gender
from genderize import Genderize

#utilize a Named Entity Recognition (NER) library to detect and remove named entities like names
#import spacy

#nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [19]:
#Load the Data into the dataframe
df = pd.read_excel('../data/INTVDATA.xlsx', sheet_name ='Main', engine ='openpyxl')

#Copy existing dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

In [20]:
#read the new .csv file
df = pd.read_csv('../data/intv_data.csv')

/var/folders/vw/1sthm_ts1dj_1hqg1xnsz19w0000gn/T/ipykernel_59052/4054036919.py:2: DtypeWarning: Columns (13,18,20,30,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/intv_data.csv')


---------

## Drop Duplicate Rows

A majority of the duplicate rows in the parent gauge dataset involve rows that are duplicates of the header row. We will use a random variable such as 'date' and remove rows that equal the name of the variable. Afterwards, we will drop additional duplicates, if any. 

In [21]:
# To make sure all duplicate "header" rows are eliminated, we pick a random column,
# remove rows where 'date' column is equal to 'date', except the first row
# Initialize a counter for deleted rows
deleted_date_rows = 0

# Iterate over the rows of the DataFrame
for i, row in df.iterrows():
    if row['date'] == 'date':
        # Remove the row if the date matches
        df.drop(i, inplace=True)
        deleted_date_rows += 1

# Count the number of rows before dropping duplicates
rows_before = len(df)

# Drop duplicate rows based on all columns except the first two, since they are indexed
result = df.drop_duplicates(subset=df.columns[2:])
        
# Count the number of rows after dropping duplicates
rows_after = len(result)

# Calculate and print the number of rows dropped
rows_dropped = (rows_before - rows_after) + deleted_date_rows
print(f"Number of duplicate rows dropped: {rows_dropped}")

#Copy existing dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

Number of duplicate rows dropped: 204


## Sample Data Frame Generation

Note: You can run this code again if you would like to reset the sample dataset.

In [29]:
#Because the main dataset is too large for data cleaning, 
#construct a small sample for faster processing. Once I am done coding, we will use the entire dataset.
#df_sample = df.sample(frac=0.1)
#df_sample.to_csv('../data/sample_data.csv', index=False)

#print("Created a sample of 10% of the total dataset")

Created a sample of 10% of the total dataset


## Main Data Cleaning

This is a summary of all the data cleaning and reformatting steps that were conducted.
- **Program** - I identified the corresponding state and county.

# Code to Clean

This section examines each variable and transforms and cleans them accordingly.

## Center

For the center, we want to correspond each center to their respective state and geographic location for future analyses.

In [36]:
unique_centers = df['center'].unique().tolist()

# Sort the list in place
unique_centers.sort()

#create a text file of the unique programs
with open('../data/unique_centers.txt', 'w') as f:
    for item in unique_centers:
        f.write("%s\n" % item)

## Date

Here, we will break down the date into three columns, seperating the year, month, and day. This code performs data cleaning on a column called 'date' in the dataset. It first checks each entry in the column for any errors in the date format, printing an error message if any are found. Then, it converts the 'date' column to a standard datetime format and creates new columns for the year, month, and day extracted from the dates. Finally, the cleaned data is saved to a CSV file. This code ensures that the dates are properly formatted, allows for easy analysis based on different time periods, and provides a clean dataset for further use.

In [22]:
## FIX ERRONEOUS DATES
# TIP: There are less than 100 erroneous rows, 96%, or most of the, 'date' entries match the same year as 'created_at'
# If the 'date' year is deemed to be erroneous, replace the 'date' year with the 'created_at' year instead

# Convert 'created_at' and 'date' columns to datetime format
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Determine current year
current_year = datetime.datetime.now().year

# Define a function to replace the year of a given date with the year of 'created_at':
def replace_year(row):
    # Extract the years
    date_year = row['date'].year
    created_at_year = row['created_at'].year
    
    # Check if the year in 'date' is below 2016 or the current year
    if date_year < 2016 or date_year > current_year:
        # Replace the year part of 'date' with 'created_at' year
        return row['date'].replace(year=created_at_year)
    else:
        return row['date']

# NOTE: Since Parent Gauge was officially implemented in 2017, anything before 2016 and past the current year should probablybe considered erroneous
# Identify erroneous dates where the year is below 2016 or above the current year
# Apply custom function to DataFrame
df['date'] = df.apply(replace_year, axis=1)

#Copy existing dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

In [21]:
# SPLIT DATE INTO MONTH, DAY, AND YEAR
# Iterate over the entries in the 'date' column
for i, date in enumerate(df['date']):
    try:
        # Try to convert the date to datetime format
        pd.to_datetime(date, format='mixed')
    except Exception:
        print(f"An error occurred at index {i} with the date: {date}")
        
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Create separate 'year', 'month', and 'day' columns
df['date_year'] = df['date'].dt.year
df['date_month'] = df['date'].dt.month
df['date_day'] = df['date'].dt.day

# save to csv
df.to_csv('../data/intv_data.csv', index=False)

## Evaluation Year

The original format of the evaluation year was formatted as '2016-2017', for instance. For easier analysis, the start and end year have been split up into two columns, "evaluation_start_year" and "evaluation_end_year." Moreover, we convert the new variables into a faster data type. Lastly, try to fix any erroneous years, then remove the rest.

In [156]:
# Get the current year
current_year = datetime.datetime.now().year

# Split the 'evaluation_year' column into two separate columns 'start_year' and 'end_year'
df[['evaluation_start_year', 'evaluation_end_year']] = df['evaluation_year'].str.split('-', expand=True)

# Convert 'evaluation_start_year' and 'evaluation_end_year to int16, handling invalid values
df['evaluation_start_year'] = pd.to_numeric(df['evaluation_start_year'], errors='coerce').astype('Int16')
df['evaluation_end_year'] = pd.to_numeric(df['evaluation_end_year'], errors='coerce').astype('Int16')

# Create intermediate 'student_enrollment_date_' column to datetime, in preparation for referencing
df['student_enrollment_date_temp'] = pd.to_datetime(df['student_enrollment_date'], format='mixed', errors='coerce')

# NOTE: Parent guage officially started in 2017, so years far from that range are erroneous.
# FIX errneous rows that were supposed to be a valid year
# Store the indices of rows with updated evaluation_start_year
updated_indices = df[(df['evaluation_start_year'] < 2016) | (df['evaluation_start_year'] > current_year)].index

# Update evaluation_start_year based on evaluation and student_enrollment_date, which is less erroneous
df.loc[df['evaluation_start_year'] < 2016, 'evaluation_start_year'] = df.loc[df['evaluation_start_year'] < 2016].apply(
    lambda row: row['student_enrollment_date_temp'].year if row['evaluation'] == 'Initial' else row['student_enrollment_date_temp'].year + 1, axis=1
)

df.loc[df['evaluation_start_year'] > current_year, 'evaluation_start_year'] = df.loc[df['evaluation_start_year'] > current_year].apply(
    lambda row: row['student_enrollment_date_temp'].year if row['evaluation'] == 'Initial' else row['student_enrollment_date_temp'].year + 1, axis=1
)

# Update evaluation_end_year only for the updated rows
df.loc[updated_indices, 'evaluation_end_year'] = df.loc[updated_indices, 'evaluation_start_year'] + 1

# Clear erroneous values where 'evaluation_start_year' is below 2016 or above current_year
df.loc[df['evaluation_start_year'] < 2016, ['evaluation_year', 'evaluation_start_year', 'evaluation_end_year']] = None
df.loc[df['evaluation_start_year'] > current_year, ['evaluation_year', 'evaluation_start_year', 'evaluation_end_year']] = None

# drop intermediate variable
df = df.drop('student_enrollment_date_temp', axis=1)

#copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

## Evaluation

Dummy variables have been created, breaking the three categorical variables into three columns.

In [48]:
#use one-hot encoding to create dummy variables in preparation for regression.
#WARNING: This, however eliminates the original 'evaluation' column
df = pd.get_dummies(df, columns=['evaluation'])

#drop excess 'evaluation_evaluation' column, if it exists.
if 'evaluation_evaluation' in df.columns:
    df.drop('evaluation_evaluation', axis=1, inplace=True)

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

## Guardian Enrollment Date

Use this variable to determine how many years the parent has been in the parent gauge program.

problem: 70% of guardian IDs have no enrollment date, but this is an important part of the analysis. need to retroactively determine what the enrollment date is. 

## Guardian, Hispanic

There are missing values, but many other variables including the guardian's native language, student's native language, whether the student is hispanic, and what language the interview was used—to determine whether the guardian was hispanic.

In [61]:
##remove anything that is not "yes" or "no"
#reconcile "dont know" and "refused" values
####pending...

# Update the guardian_hispanic column
df.loc[~df["guardian_hispanic"].isin(["Yes", "No"]), "guardian_hispanic"] = ""

#check the column "guardian_native_language" "student_hispanic", 
#"student_native_language", "language of interview", 
# Define a function to fill missing values in guardian_hispanic column based on conditions
def fill_guardian_hispanic(row):
    if pd.isnull(row['guardian_hispanic']) or row['guardian_hispanic'] not in ['Yes', 'No']:
        if row['student_hispanic'] == 'Yes':
            return 'Yes'
        elif row['student_hispanic'] == 'No':
            return 'No'
        elif row['guardian_native_language'] == 'Spanish':
            return 'Yes'
        elif row['student_native_language'] == 'Spanish':
            return 'Yes'
        elif row['language'] == 'Spanish':
            return 'Yes'
        else:
            return 'No'
    else:
        return row['guardian_hispanic']

# Apply the function to fill missing values in guardian_hispanic column
df['guardian_hispanic'] = df.apply(fill_guardian_hispanic, axis=1)

# Convert "yes" and "no" to binary dummy variables
df['guardian_hispanic'] = df['guardian_hispanic'].map({'Yes': True, 'No': False})

# Write the updated data to a new CSV file
df.to_csv('../data/intv_data.csv', index=False)

## Guardian Native Language

7% of rows for guardian native language are missing, While the student's native language is not necessarily the same as the guardian's native language, we might be able to reference for statistical purposes. This will cut down missing to 2.5%. 91% are in English and almost 7% are in Spanish. Imputing the values with modal value (English) does not drastically change the numbers. 

In [93]:
# Fill in missing values
# Reference 'language' of interview', if interview was done in Spanish, assume guardian's native language is Spanish
df.loc[df['guardian_native_language'].isnull() & (df['language'] == 'Spanish'), 'guardian_native_language'] = 'Spanish'

# Then reference 'student_native_language' to fill in other missing values
df['guardian_native_language'] = df['guardian_native_language'].fillna(df['student_native_language'])

# Afterwards, reference 'guardian_id' as well
df['guardian_native_language'] = df['guardian_native_language'].fillna(df.groupby('guardian_id')['guardian_native_language'].transform('first'))

#Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

In [ ]:
# consider grouping remaining languages into one category

## Guardian Race

tasks: 

In [48]:
######## clean and normalize
# Convert to lowercase
# Remove leading and trailing whitespace
# Remove values that start with ', '
df['guardian_race'] = df['guardian_race'].str.lower().str.strip().str.replace('^, ', '', regex=True)

In [50]:
# Function to simplify categories
def simplify_race_category(category):
    # Check for NaN (missing value)
    if pd.isna(category):
        return "Unknown"
        
    # Convert category to lowercase for case-insensitive comparison
    category = category.lower()
    
    # Count how many specific categories are in the value
    count = 0
    if "white" in category:
        count += 1
    if "black" in category or "african american" in category:
        count += 1
    if "asian" in category:
        count += 1
    if "native" in category or "american indian" in category or "alaska native" in category:
        count += 1
    if "hawaiian" in category or "pacific islander" in category:
        count += 1
    if "other" in category:
        count += 1
    if "don't know" in category or "unknown" in category:
        count += 0
    
    # Based on count, return the simplified category
    if count == 0:
        return "Unknown"
    elif count == 1:
        if "white" in category:
            return "White"
        if "black" in category or "african american" in category:
            return "Black"
        if "asian" in category:
            return "Asian"
        if "native" in category or "american indian" in category or "alaska native" in category:
            return "Native American or Alaska Native"
        if "hawaiian" in category or "pacific islander" in category:
            return "Native Hawaiian or Pacific Islander"
        if "other" in category:
            return "Other"
    else:
        return "Multiracial"

# Apply the simplification function to the 'guardian_race' column
df['guardian_race'] = df['guardian_race'].apply(simplify_race_category)

In [52]:
# Group by guardian_id and fill missing values in guardian_race
df['guardian_race'] = df.groupby('guardian_id')['guardian_race'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [ ]:
# Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

## Guardian Date of Birth

use guardian's DOB and interview year to determine guardian’s age during time of interview. We create a new variable named 'guardian_age'

In [24]:
# use guardian's DOB and interview year to determine guardian’s age during time of interview. 
# CORRECT erroneous 'guardian_birth_date' and 'date' variable first before determining age

# Step 1: Convert 'guardian_birth_date' and 'date' columns to datetime
df['guardian_birth_date'] = pd.to_datetime(df['guardian_birth_date'], errors='coerce')
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Step 2: try to map guardian_birth_date with corresponding guardian_id
# Create a dictionary mapping guardian_ids to non-missing birth dates
mapping_dict = df.dropna(subset=['guardian_birth_date']).set_index('guardian_id')['guardian_birth_date'].to_dict()

# Fill missing birth dates using the mapping dictionary
df['guardian_birth_date'] = df['guardian_id'].map(mapping_dict).fillna(df['guardian_birth_date'])

# Filter rows where guardian_birth_date is out of bounds
out_of_bounds_dates = df[(df['guardian_birth_date'].dt.year < 1800) | (df['guardian_birth_date'].dt.year > 2100)]

# Remove rows with out-of-bounds dates
df.drop(out_of_bounds_dates.index, inplace=True)


# Step 3: Calculate the age at the time of the interview
df['guardian_age'] = pd.NaT  # NEW VARIABLE: Initialize the column with missing values
for i, row in df.iterrows():
    try:
        age = (row['date'] - row['guardian_birth_date']).days // 365
        df.at[i, 'guardian_age'] = age
    except:
        pass  # Ignore any dates that are out of bounds

## Note: certain date of births are observed to be precisely 100 years off, 
        #try adding or subtracting 100 years to see if the age then becomes valid
# Step 4: if guardian_age is below 18 or above 99, see if subtracting or adding 100 years will correct the 
    # 'guardian"age' (making it between 18 and 99). 
    # if it does, update 'guardian_birth_date' and 'guardian_age' with the corresponding modification.
# Filter rows where guardian_age is below 18 or above 99
filtered_df = df[(df['guardian_age'] < 18) | (df['guardian_age'] > 99)]

# Iterate over each row
for index, row in filtered_df.iterrows():
    birth_date = row['guardian_birth_date']
    original_age = row['guardian_age']

    # Subtract or add 100 years to correct age
    modified_birth_date_minus = birth_date - timedelta(days=36525)  # Subtracting 100 years
    modified_birth_date_plus = birth_date + timedelta(days=36525)  # Adding 100 years

    modified_age_minus = df.at[index, 'guardian_age'] - 100
    modified_age_plus = df.at[index, 'guardian_age'] + 100

    # Check if the modified ages fall within a reasonable and valid age range
    if 18 <= modified_age_minus <= 65:
        df.at[index, 'guardian_birth_date'] = modified_birth_date_minus
        df.at[index, 'guardian_age'] = modified_age_minus
    elif 18 <= modified_age_plus <= 65:
        df.at[index, 'guardian_birth_date'] = modified_birth_date_plus
        df.at[index, 'guardian_age'] = modified_age_plus

# Step 5: Clear 'guardian_age' and 'guardian_birth_date' where 'guardian_age' 
        #values are less than 18 or above 99 -- dont remove row, clear field
filtered_rows = df[(df['guardian_age'] < 18) | (df['guardian_age'] > 99)]

# Update 'guardian_age' and 'guardian_birth_date' with null/missing values
df.loc[filtered_rows.index, ['guardian_age', 'guardian_birth_date']] = None


# Last step: Write the updated dataframe to a new CSV file
df.to_csv('../data/intv_data.csv', index=False)

## Guardian Sex

Here, we use the gender guesser to fill in missing values. 

In [25]:
#Normalize gender values for more proper analysis
df['guardian_sex'] = df['guardian_sex'].replace('F', 'Female')
df['guardian_sex'] = df['guardian_sex'].replace('M', 'Male')
df['guardian_sex'] = df['guardian_sex'].replace('Unknown', '')

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

In [30]:
# use both the "gender_guesser" and "Genderize" library to fill in missing values
# warning: Genderize relies on HTTP requests, so it may lag

# Create a gender detector
detector = gender.Detector()

# Function to fill missing gender
def fill_missing_gender(row):
    # First try to fill in from existing data in different rows
    if row['guardian_id'] in guardian_sex_map:
        return guardian_sex_map[row['guardian_id']]
    
    # Secondary method - guess gender using gender_guesser
    name = row['guardian'].split()[0]  # assuming first name is the first part
    guessed_gender = detector.get_gender(name)
    
    # Map guessed gender to your gender categories
    if guessed_gender in ['male', 'mostly_male']:
        return 'Male'
    elif guessed_gender in ['female', 'mostly_female']:
        return 'Female'
    
    # Tertiary method - use genderize.io
    else:
        guessed_gender = Genderize().get([name])[0]['gender']
        if guessed_gender == 'male':
            return 'Male'
        elif guessed_gender == 'female':
            return 'Female'
        else:
            return 'Unknown'

In [27]:
# FILL IN MISSING VALUES: Guess the gender using customized function
# Creating a mapping from guardian_id to guardian_sex
guardian_sex_map = df.dropna(subset=['guardian_sex']).groupby('guardian_id')['guardian_sex'].first().to_dict()

# Select rows where guardian_sex is missing
missing_gender_indices = df[df['guardian_sex'].isna()].index

# Apply the function only to rows with missing guardian_sex
df.loc[missing_gender_indices, 'guardian_sex'] = df.loc[missing_gender_indices].apply(fill_missing_gender, axis=1)

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

In [85]:
# Using direct mapping to create dummy variable out of guardian_sex
df['guardian_female'] = np.where(df['guardian_sex'] == 'Female', True, False)

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

## Language of Interview

There are only less than 50 missing or invalid variables. As most interviews were done in the english language, impute variable with "English."

In [45]:
#Assume interview was done in english. Fill empty values with "English"
df['language'].fillna("English", inplace=True)

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

## Mode of Interview

convert to dummy variable

In [14]:
#convert to dummy variable
# Using direct mapping to create dummy variable out of mode
df['phone_interview'] = np.where(df['mode'] == 'Phone', True, False)

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

## Program

## Student Enrollment Date

Erroneous years range from 1915 to 2121. need to fix them, and reference other columns. 

In [27]:
# Convert 'student_enrollment_date' to datetime format
df['student_enrollment_date'] = pd.to_datetime(df['student_enrollment_date'], errors='coerce')

#fix erroneous and missing years

## Student Disability Status

*If entry is blank, change to 'None' if interview disregards questions intended for parents of students with disabilities. 

In [17]:
#convert disability questions from string to numeric data type
df['QD1'] = pd.to_numeric(df['QD1'], errors='coerce')
df['QD1a'] = pd.to_numeric(df['QD1a'], errors='coerce')
df['QD2'] = pd.to_numeric(df['QD2'], errors='coerce')
df['QD2a'] = pd.to_numeric(df['QD2a'], errors='coerce')

# Custom function to check if any of the disability question columns have value between 1 to 5
def has_disability(row):
    return 1 <= row['QD1'] <= 5 or 1 <= row['QD1a'] <= 5 or 1 <= row['QD2'] <= 5 or 1 <= row['QD2a'] <= 5

# Apply the custom function and fill in the missing values in 'student_has_disability'
df['student_has_disability'] = df.apply(lambda row: has_disability(row) if pd.isna(row['student_has_disability']) else row['student_has_disability'], axis=1)

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

## Student, Hispanic

remove anything that is not "yes" or "no". reconcile "dont know" and "refused" values.

In [13]:
##remove anything that is not "yes" or "no", and reconcile "dont know" and "refused" values

# Update the guardian_hispanic column
df.loc[~df["guardian_hispanic"].isin(["Yes", "No"]), "guardian_hispanic"] = ""

#check the column "guardian_native_language" "guardian_hispanic", "student_native_language"
# Define a function to fill missing values in student_hispanic column based on conditions
def fill_student_hispanic(row):
    if pd.isnull(row['student_hispanic']):
        if row['guardian_hispanic'] == 'Yes':
            return 'Yes'
        elif row['guardian_hispanic'] == 'No':
            return 'No'
        elif row['guardian_native_language'] == 'Spanish':
            return 'Yes'
        elif row['student_native_language'] == 'Spanish':
            return 'Yes'
        elif pd.notnull(row['student_native_language']):
            return 'No'
        else:
            return ''
    else:
        return row['student_hispanic']

# Apply the function to fill missing values in student_hispanic column
df['student_hispanic'] = df.apply(fill_student_hispanic, axis=1)

# Convert "yes" and "no" to binary dummy variables
#df = pd.get_dummies(df, columns=["guardian_hispanic"], prefix="guardian_hispanic", drop_first=True)

# Write the updated data to a new CSV file
df.to_csv('../data/intv_data.csv', index=False)

In [ ]:
#if student_race is missing, consider using

## Student Birth Date

In [38]:
# use student's DOB and interview year to determine student’s age during time of their guardian's interview. 
# CORRECT erroneous 'student_birth_date' and 'date' variable first before determining age

# Step 1: Convert 'student_birth_date' and 'date' columns to datetime
df['student_birth_date'] = pd.to_datetime(df['student_birth_date'], errors='coerce')
# df['date'] = pd.to_datetime(df['date'], errors='coerce') <---if necessary

# Step 2: try to map student_birth_date with corresponding student_id
# Create a dictionary mapping student_id's to non-missing birth dates
mapping_dict = df.dropna(subset=['student_birth_date']).set_index('student_id')['student_birth_date'].to_dict()

# Fill missing birth dates using the mapping dictionary
df['student_birth_date'] = df['student_id'].map(mapping_dict).fillna(df['student_birth_date'])

#Step 3:  Filter rows where guardian_birth_date is out of bounds
out_of_bounds_dates = df[(df['student_birth_date'].dt.year < 1800) | (df['student_birth_date'].dt.year > 2100)]

# Remove rows with out-of-bounds dates
df.drop(out_of_bounds_dates.index, inplace=True)


# Step 4: Calculate the age at the time of their guardian's interview
df['student_age'] = pd.NaT  # NEW VARIABLE: Initialize the column with missing values
for i, row in df.iterrows():
    try:
        age = (row['date'] - row['student_birth_date']).days // 365
        df.at[i, 'student_age'] = age
    except:
        pass  # Ignore any dates that are out of bounds

## Note: certain date of births are observed to be precisely 100 years off, 
        #try adding or subtracting 100 years to see if the age then becomes valid
# Step 5: if student is below 0 or above 6, see if subtracting or adding 100 years will correct the 
    # 'student "age' (making it between 0 and 6). 
    # if it does, update 'student_birth_date' and 'student_age' with the corresponding modification.
# Filter rows where student_age is below 0 or above 6
filtered_df = df[(df['student_age'] < 0) | (df['student_age'] > 6)]

# Iterate over each row
for index, row in filtered_df.iterrows():
    birth_date = row['student_birth_date']
    original_age = row['student_age']

    # Subtract or add 100 years to correct age
    modified_birth_date_minus = birth_date.replace(year=birth_date.year - 100)  # Subtracting 100 years
    modified_birth_date_plus = birth_date.replace(year=birth_date.year + 100)  # Adding 100 years

    modified_age_minus = df.at[index, 'student_age'] - 100
    modified_age_plus = df.at[index, 'student_age'] + 100

    # Check if the modified ages fall within a reasonable and valid age range
    if 0 <= modified_age_minus <= 6:
        df.at[index, 'student_birth_date'] = modified_birth_date_minus
        df.at[index, 'student_age'] = modified_age_minus
    elif 0 <= modified_age_plus <= 6:
        df.at[index, 'student_birth_date'] = modified_birth_date_plus
        df.at[index, 'student_age'] = modified_age_plus

# Step 6: Clear 'student_age' and 'student_birth_date' where 'student_age' 
        #values are less than 0 or above 6 -- dont remove row, clear field
filtered_rows = df[(df['student_age'] < 0) | (df['student_age'] > 6)]

# Update 'student-age' and 'student_birth_date' with null/missing values
df.loc[filtered_rows.index, ['student_age', 'student_birth_date']] = None

# Step 7: also calculate the student's age in months
# Calculate age in months using a reference date
df['student_age_months'] = ((df['date'] - df['student_birth_date']) / pd.Timedelta(days=30.44))
df['student_age_months'] = df['student_age_months'].where(df['student_age_months'].notnull() & np.isfinite(df['student_age_months']), np.nan).astype(float)

# Last step: Write the updated dataframe to a new CSV file
df.to_csv('../data/intv_data.csv', index=False)

## Student in last year

41% missing values, drop

In [167]:
# FILL IN MISSING VALUES

###########
# Reference variable 'Q1', which is a question intended for first-timers. if values are 1 and above, i.e. answered, assume that student_in_last_year equals to false.
# Convert 'Q1' to numeric, invalid values will be set as NaN
df['Q1'] = pd.to_numeric(df['Q1'], errors='coerce')

# fill in missing values -> this effectively cuts missing values from 41% to 17%.
df.loc[df['Q1'] > 0, 'student_in_last_year'] = False
df.loc[df['Q1'] == -3, 'student_in_last_year'] = True # if question was skipped conditionally, it assumes student was enrolled the previous year

###########
# Fill missing values in 'student_in_last_year' based on matching 'student_id' and 'evaluation_start_year' -> effectively drops from 17% to 5.5%.
df['student_in_last_year'] = df.groupby(['student_id', 'evaluation_start_year'])['student_in_last_year'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

###########
# check if there are past entries for each student_id' where there is an entry in the previous evaluation year
# Custom function to check if evaluation_start_year - 1 exists for the same student_id
def check_previous_year(row):
    if pd.isna(row['student_in_last_year']):
        prev_year = row['evaluation_start_year'] - 1
        exists = df[(df['student_id'] == row['student_id']) & (df['evaluation_start_year'] == prev_year)].shape[0] > 0
        if exists:
            return True
    return row['student_in_last_year']

# Apply the custom function to each row
df['student_in_last_year'] = df.apply(check_previous_year, axis=1)

# Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

## Student Native Language

Apply similar steps as Guardian's native language for statistical purposes. Less than 5% of values are missing, and assuming native language by guardian's native language should not drastically affect analysis or introduce bias. 

In [98]:
# Fill in missing values
# Reference 'guardian_native_language' to fill in missing values
df['student_native_language'] = df['student_native_language'].fillna(df['guardian_native_language'])

# Afterwards, reference 'student_id' as well
df['student_native_language'] = df['student_native_language'].fillna(df.groupby('student_id')['student_native_language'].transform('first'))

#Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

In [ ]:
# consider grouping languages other than English and Spanish together

## Student Program Type

For missing values, we can check the center's program type. Apply modal value of assigned programif above 90% frequency threshold. Then, need to deal with variables that state both "head start" and "early head start." Can possibly also base it off student age. 7% is missing.

In [23]:
# Function to calculate mode and check if its frequency is above 80%
def modal_program_to_use(ser):
    mode_count = ser.value_counts()
    total = len(ser.dropna())
    if total > 0 and (mode_count.iloc[0] / total) > 0.80:
        return mode_count.index[0]
    else:
        return None

# Compute the modes that satisfy the condition for each center
modes_to_use_by_center = df.groupby('center')['student_program_type'].transform(modal_program_to_use)

# Replace missing values in 'student_service_type' column based on center's mode
df['student_program_type'].fillna(modes_to_use_by_center, inplace=True)

# Next, compute the modes that satisfy the condition for each program
modes_to_use_by_program = df.groupby('program')['student_program_type'].transform(modal_program_to_use)

# Replace remaining missing values in 'student_service_type' column based on program's mode
df['student_program_type'].fillna(modes_to_use_by_program, inplace=True)

# Fill the remaining missing values with the most frequent category overall, for analytical purposes
most_frequent_program = df['student_program_type'].mode().iloc[0]
df['student_program_type'].fillna(most_frequent_program, inplace=True)

# Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

## Student Race

#there are too many combinations, over 85 of them. need to simplify and create dummy variables. If student_race is missing, consider using guardian variable (for statistical purposes). cannot assume individually.

In [56]:
######## clean and normalize
# Convert to lowercase
# Remove leading and trailing whitespace
# Remove values that start with ', '
df['student_race'] = df['student_race'].str.lower().str.strip().str.replace('^, ', '', regex=True)

In [59]:
# Function to simplify categories
def simplify_race_category(category):
    # Check for NaN (missing value)
    if pd.isna(category):
        return "Unknown"
        
    # Convert category to lowercase for case-insensitive comparison
    category = category.lower()
    
    # Count how many specific categories are in the value
    count = 0
    if "white" in category:
        count += 1
    if "black" in category or "african american" in category:
        count += 1
    if "asian" in category:
        count += 1
    if "native" in category or "american indian" in category or "alaska native" in category:
        count += 1
    if "hawaiian" in category or "pacific islander" in category:
        count += 1
    if "other" in category:
        count += 1
    if "don't know" in category or "unknown" in category:
        count += 0
    
    # Based on count, return the simplified category
    if count == 0:
        return "Unknown"
    elif count == 1:
        if "white" in category:
            return "White"
        if "black" in category or "african american" in category:
            return "Black"
        if "asian" in category:
            return "Asian"
        if "native" in category or "american indian" in category or "alaska native" in category:
            return "Native American or Alaska Native"
        if "hawaiian" in category or "pacific islander" in category:
            return "Native Hawaiian or Pacific Islander"
        if "other" in category:
            return "Other"
    else:
        return "Multiracial"

# Apply the simplification function to the 'student_race' column
df['student_race'] = df['student_race'].apply(simplify_race_category)

# Group by student_id and fill missing values in student_race
df['student_race'] = df.groupby('student_id')['student_race'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [61]:
# consider filling unknown student race values to that of their guardian, (for statistical purposes)
mask = (df['student_race'] == 'Unknown') & (df['guardian_race'] != 'Unknown')
df.loc[mask, 'student_race'] = df.loc[mask, 'guardian_race']

In [63]:
# Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

## Student Service Type

Try to reference off missing variables like program or center. 

In [78]:
# Function to calculate mode and check if its frequency is above 75%
def mode_to_use(ser):
    mode_count = ser.value_counts()
    total = len(ser.dropna())
    if total > 0 and (mode_count.iloc[0] / total) > 0.75:
        return mode_count.index[0]
    else:
        return None

# Compute the modes that satisfy the condition for each center
modes_to_use_by_center = df.groupby('center')['student_service_type'].transform(mode_to_use)

# Replace missing values in 'student_service_type' column based on center's mode
df['student_service_type'].fillna(modes_to_use_by_center, inplace=True)

# Next, compute the modes that satisfy the condition for each program
modes_to_use_by_program = df.groupby('program')['student_service_type'].transform(mode_to_use)

# Replace remaining missing values in 'student_service_type' column based on program's mode
df['student_service_type'].fillna(modes_to_use_by_program, inplace=True)

# Fill the remaining missing values with the most frequent category overall, for analytical purposes
most_frequent_category = df['student_service_type'].mode().iloc[0]
df['student_service_type'].fillna(most_frequent_category, inplace=True)

# Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

## Student Sex

use gender_guesser again

In [31]:
def fill_missing_gender_student(row):
    # First try to fill in from existing data in different rows
    if row['student_id'] in student_sex_map:
        return student_sex_map[row['student_id']]
    
    # Secondary method - guess gender using gender_guesser
    name = row['student'].split()[0]  # assuming first name is the first part
    guessed_gender = detector.get_gender(name)
    
    # Map guessed gender to your gender categories
    if guessed_gender in ['male', 'mostly_male']:
        return 'Male'
    elif guessed_gender in ['female', 'mostly_female']:
        return 'Female'
    
    # Tertiary method - use genderize.io
    else:
        guessed_gender = Genderize().get([name])[0]['gender']
        if guessed_gender == 'male':
            return 'Male'
        elif guessed_gender == 'female':
            return 'Female'
        else:
            return 'Unknown'

In [32]:
#Normalize gender values for more proper analysis
df['student_sex'] = df['student_sex'].replace('F', 'Female')
df['student_sex'] = df['student_sex'].replace('M', 'Male')
df['student_sex'] = df['student_sex'].replace('Unknown', '')

In [34]:
# FILL IN MISSING VALUES: Guess the gender using customized function
# Creating a mapping from guardian_id to guardian_sex
student_sex_map = df.dropna(subset=['student_sex']).groupby('student_id')['student_sex'].first().to_dict()

# Select rows where guardian_sex is missing
missing_gender_indices_student = df[df['student_sex'].isna()].index

# Apply the function only to rows with missing guardian_sex
df.loc[missing_gender_indices_student, 'student_sex'] = df.loc[missing_gender_indices_student].apply(fill_missing_gender_student, axis=1)

In [ ]:
# Using direct mapping to create dummy variable out of guardian_sex
df['student_female'] = np.where(df['student_sex'] == 'Female', True, False)

#save updates to csv
df.to_csv('../data/intv_data.csv', index=False)

## Student Was Early Headstart

In [56]:
# Convert string representations of 'True' and 'False' to boolean
df['student_was_early_head_start'] = df['student_was_early_head_start'].replace({'False': False, 'True': True})

# Write the updated dataframe to a new CSV file
df.to_csv('../data/intv_data.csv', index=False)

## Student Was Head Start

In [57]:
# Convert string representations of 'True' and 'False' to boolean
df['student_was_head_start'] = df['student_was_head_start'].replace({'False': False, 'True': True})

# Write the updated dataframe to a new CSV file
df.to_csv('../data/intv_data.csv', index=False)

# Next Section: Likert Scale Interview Questions

### CHANGE ALL UNANSWERED VALUES TO BLANK

In [22]:
##CHANGE ALL UNANSWERED VALUES TO BLANK##
#determine all of the likert scale interview variables

variables = ['Q1', 'Q1a', 'Q1b', 'Q1c', 'Q1d', 'Q2', 'Q2a', 'Q3', 'Q3a', 'Q4', 'Q4a', 'Q5', 'Q5a', 'Q6', 'Q6a',
             'Q7', 'Q7a', 'Q8', 'Q8a', 'QD1', 'QD1a', 'QD2', 'QD2a', 'Q9', 'Q9a', 'Q10', 'Q10a', 'Q11', 'Q11a',
             'Q12', 'Q12a', 'Q13', 'Q13a', 'Q14', 'Q14a', 'Q15', 'Q15a', 'Q16', 'Q16a', 'Q17', 'Q17a', 'Q18',
             'Q18a', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25']

#convert string values to numeric, change the datatype to int
df[variables] = df[variables].apply(pd.to_numeric, errors='coerce', downcast='integer')

#replace the row values with blanks if the value is below 0
df[variables] = df[variables].applymap(lambda x: '' if x < 0 else x)

#Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

### DROP ALL BLANK INTERVIEWS

In [23]:
# Select columns starting with 'Q'
q_columns = [col for col in df.columns if col.startswith('Q')]

# Count the number of rows where all 'Q' columns are blank or null
count_all_blank = len(df[(df[q_columns].isnull() | (df[q_columns] == '')).all(axis=1)])

# Calculate the percentage of rows where all 'Q' columns are blank
percentage_all_blank = (count_all_blank / len(df)) * 100

# Delete the rows where all 'Q' columns are blank
df = df[~(df[q_columns].isnull() | (df[q_columns] == "")).all(axis=1)]

# Print the number and percentage of deleted rows
print("Number of rows deleted where all 'Q' variables are blank:", count_all_blank)
print("Percentage of rows deleted where all 'Q' variables are blank: {:.2f}%".format(percentage_all_blank))

#Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

Number of rows deleted where all 'Q' variables are blank: 20470
Percentage of rows deleted where all 'Q' variables are blank: 10.00%


## Imputation of unanswered responses

https://www.researchgate.net/post/How-to-handle-missing-values-in-a-likert-scale-type-survey

You can handle this situation by applying following techniques.
1. Neutral-value substitution: You can use neutral value of the likert scale. In your case neutral value would be 3.5 but if your overall average score is less than 3.5 then you could not able to use this method.
2. Mean-value substitution: If the average response for your actual respondents is 2.67, then substitute 2.67 for all your missing values. If it’s 5, use 5. But be aware that it is applicable only when the number of respondents with missing data and the number of items missing were 20% or less. More here
Article Missing Data in Likert Ratings: A Comparison of Replacement Methods
3. Imputation: You can calculate those values by the help of SAS and SPSS by using Approximate Bayesian bootstrap with Propensity score. More here
Article On the Imputation of Missing Data in Surveys with Likert-Type Scales

In [85]:
## need to consider how questions are intended to not be answered based on evaluation period and circumstances (e.g. student disability), filter them

Count of missing values in 'Q8a': 529
Percentage of missing values in 'Q8a': 3.88%


In [ ]:
# completion rates

# LIKERT SUM CALCULATION

Calculate a likert sum for statistical analyses

In [24]:
#likert sum for initial interviews
#exclude Q10-Q11 due to non-response rate
variable_questions_initial = ['Q2', 'Q3', 'Q4', 'Q5',
                      'Q6', 'Q7', 'Q8', 'Q9', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18']

# Calculate the Likert Sum for rows where evaluation is 'Initial'
df['likert_sum_initial'] = df[df['evaluation'] == 'Initial'][variable_questions_initial].sum(axis=1)

#likert sum for mid/post interviews
#exclude Q10-Q11 because of non-response rate
variable_questions_midpost = ['Q2', 'Q2a', 'Q3', 'Q3a', 'Q4', 'Q4a', 'Q5', 'Q5a',
                      'Q6', 'Q6a', 'Q7', 'Q7a', 'Q8', 'Q8a', 'Q9', 'Q9a',
                      'Q12', 'Q12a', 'Q13', 'Q13a',
                      'Q14', 'Q14a', 'Q15', 'Q15a', 'Q16', 'Q16a', 'Q17', 'Q17a',
                      'Q18', 'Q18a', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25']

# Calculate the Likert Sum of mid/post questions for rows where evaluations are 'Midyear' and 'Post'
df['likert_sum_mid_post'] = df[df['evaluation'].isin(['Midyear', 'Post'])][variable_questions_midpost].sum(axis=1)

# Calculate the Likert Sum for initial questions for rows where evaluations are 'Midyear' and 'Post'
df['likert_sum_mid_post_initial'] = df[df['evaluation'].isin(['Midyear', 'Post'])][variable_questions_initial].sum(axis=1)

#Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

/var/folders/vw/1sthm_ts1dj_1hqg1xnsz19w0000gn/T/ipykernel_44653/2388527200.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['likert_sum_initial'] = df[df['evaluation'] == 'Initial'][variable_questions_initial].sum(axis=1)
/var/folders/vw/1sthm_ts1dj_1hqg1xnsz19w0000gn/T/ipykernel_44653/2388527200.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['likert_sum_mid_post'] = df[df['evaluation'].isin(['Midyear', 'Post'])][variable_questions_midpost].sum(axis=1)
/var/folders/vw/1sthm_ts1dj_1hqg1xnsz19w0000gn/T/ipykernel_44653/2388527200.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprec

## Survey Completion Rate

In [25]:
## Survey completion rate
# Function to calculate completion rate as a proportion
def calculate_completion_rate_as_proportion(row):
    if row['evaluation'] == 'Initial':
        variable_questions = variable_questions_initial
    elif row['evaluation'] in ['Midyear', 'Post']:
        variable_questions = variable_questions_midpost
    else:
        return np.nan
    
    completed_count = row[variable_questions].notnull().sum()
    total_count = len(variable_questions)
    
    return completed_count / total_count

# Apply the function to each row
df['completion_rate'] = df.apply(calculate_completion_rate_as_proportion, axis=1)

#Copy updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

## Calculate Normalized Likert Sum Difference Score

# Next Section: Open Interview Questions

perhaps we can use data analysis to see how sentiments change
https://www.surveypractice.org/article/25699-what-to-do-with-all-those-open-ended-responses-data-visualization-techniques-for-survey-researchers

In [61]:
#first need to translate spanish-language to english
columns_to_translate = ['OQ1', 'OQ2', 'OQ3', 'OQ3a', 'OQ4', 'OQ5', 'OQ6', 'OQ7', 'OQ8', 'OQ9', 'OQ10']

In [95]:
#SCRUB ALL NAMES WITH GENERIC NAMES
# Load the spaCy multilingual language model
nlp = spacy.load('xx_ent_wiki_sm')

# Define the generic placeholder for student names
generic_name = "Student"

# Define a function to remove names from a text string
def remove_names(text):
    doc = nlp(text)
    cleaned_text = ' '.join([token.text if token.ent_type_ != 1 else generic_name for token in doc])
    return cleaned_text

# Convert float values to strings in the 'OQ1' column
df['OQ1'] = df['OQ1'].astype(str)

# Apply the function to the 'OQ1' column
df['OQ1_cleaned'] = df['OQ1'].apply(remove_names)

# Print the updated column
print(df['OQ1_cleaned'])

0         YES I DO THE STORY THAT I HAVE SINCE MY OLD CH...
1                                                       nan
2         I am more open with telling the staff any prob...
3         it has helped me and my children . I have lear...
4         I mean my relationship with Stacy , its a good...
                                ...                        
204670    This is our last summer with the program .   W...
204671    Mantuvo conversaciones , escucho acerca de nue...
204672    La maestra hizo preguntas acerca de mi familia...
204673    Se intereso por mi familia y nuestras necesida...
204674    Manteniendo conversaciones , haciendo pregunta...
Name: OQ1_cleaned, Length: 204675, dtype: object


## Interview Text Normalization

In [89]:
def normalize_text(text):
    text = text.lower() # converts all interview responses to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # removes special characters and punctuation
    text = text.strip() # normalizes text e.g. extra spaces
    # Apply additional normalization steps as needed
    return text

for column in df.columns:
    if column.startswith('OQ'):
        df[column] = df[column].apply(normalize_text)
        

#TOKENIZATION
#REMOVING STOPWORDS
#CORRECTING MISSPELLED WORDS
#STEMMING AND LEMATIZATION
#HANDLE MISSING VALUES

###clear interview responses where text says "refused to answer" "don't know" or "parent refused interview"

# Write the updated dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

# Remove Unnecessary Columns

After meticulously going through the data cleaning process, it is important to ensure that the dataset is streamlined and efficient for analysis. Removing excess columns that do not contribute to the analysis or could introduce noise is a critical step in data preparation. This step not only enhances the readability and manageability of the dataset but also optimizes memory usage and potentially speeds up processing time. It’s vital to scrutinize each column and ascertain its relevance in context to the analysis goals.

In [31]:
## REMOVE IMPRACTICAL, UNNECESSARY, PRIVATE, AND NO LONGER NEEDED COLUMNS
# List of columns to be removed
columns_to_remove = ['created_at', 'guardian', 'guardian_employment', 'guardian_highest_education', 'guardian_birth_date', 'evaluation_year',
                     'guardian_sex', 'guardian_vendor_id', 'interview_id', 'interviewer_id', 'mode', 'student_sex',
                     'student_was_early_head_start', 'student_was_head_start', # remove these two, high percent missing values
                     'interviewer', 'interviewer_vendor_id', 'student', 'student_staff_vendor_id', 'student_birth_date',
                     'student_vendor_id', 'student_staff', 'student_staff_id']

# Removing the columns from the DataFrame
df = df.drop(columns=columns_to_remove)

#save updates to working csv
df.to_csv('../data/intv_data.csv', index=False)

# Data Type Reconfiguration

To make the analysis faster.

In [194]:
# Infer and assign data types based on majority or inferred data types
df_inferred = df.infer_objects()